In [22]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   

# Load data

In [23]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs_bin.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)

# all edges ON (example); add node/edge 0s as needed
states = {eid: 1 for eid in edges.keys()}

k_val, status, _ = fbg.eval_global_conn_k(states, G_base, target_k=2)
print("k =", k_val, "status =", status)

k = 1 status = f


In [24]:
row_names = list(edges.keys()) + ['sys']
n_state = 2 # binary states

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names[:-1]]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

In [25]:
def s_fun(comps_st):
    travel_time, sys_st, info = fbg.eval_travel_time_to_nearest(comps_st, G_base, origin, dests,
                                                         avg_speed=60, # km/h
                                                         target_max = 0.5, # hours: it shouldn't take longer than this compared to the original travel time
                                                         length_attr = 'length_km')
    if sys_st == 's':
       path = info['path_filtered_edges'] 
       min_comps_st = {eid: ('>=', 1) for eid in path} # edges in the path are working
       min_comps_st['sys'] = ('>=', 1) # system edge is also working
    else:
        min_comps_st = None
    return travel_time, sys_st, min_comps_st

In [26]:
origin = 'n1'
dests = ['n22', 'n66']

In [27]:
TSUMPATH = Path("tsum_res") 

rules_mat_surv = torch.load(TSUMPATH / "rules_surv.pt", map_location="cpu")
rules_mat_surv = rules_mat_surv.to(device)
rules_mat_fail = torch.load(TSUMPATH / "rules_fail.pt", map_location="cpu")
rules_mat_fail = rules_mat_fail.to(device)

C:\Users\jb622s\AppData\Local\Temp\ipykernel_13016\2870499433.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rules_mat_surv = torch.load(TSUMPATH / "rules_surv.pt", map

# Calculate system probabilities

## Marginal probability

In [28]:
pr_cond = tsum.get_comp_cond_sys_prob(
    rules_mat_surv,
    rules_mat_fail,
    probs,
    comps_st_cond = {},
    row_names = row_names,
    s_fun = s_fun
)
print(f"P(sys=1) = {pr_cond['survival']:.3e}")
print(f"P(sys=0 ) = {pr_cond['failure']:.3e}\n")

P(sys=1) = 9.962e-01
P(sys=0 ) = 3.844e-03



## Conditional probability given "one" component's survival

In [29]:
for x in row_names[:-1]:
    print(f"Eval P(sys | {x}=1)")
    pr_cond = tsum.get_comp_cond_sys_prob(
        rules_mat_surv,
        rules_mat_fail,
        probs,
        comps_st_cond = {x: 1},
        row_names = row_names,
        s_fun = s_fun
    )
    print(f"P(sys=1 | {x}=1) = {pr_cond['survival']:.3e}")
    print(f"P(sys=0 | {x}=1) = {pr_cond['failure']:.3e}\n")

Eval P(sys | e0001=1)
P(sys=1 | e0001=1) = 9.974e-01
P(sys=0 | e0001=1) = 2.607e-03

Eval P(sys | e0002=1)
P(sys=1 | e0002=1) = 9.972e-01
P(sys=0 | e0002=1) = 2.769e-03

Eval P(sys | e0003=1)
P(sys=1 | e0003=1) = 9.972e-01
P(sys=0 | e0003=1) = 2.792e-03

Eval P(sys | e0004=1)
P(sys=1 | e0004=1) = 9.961e-01
P(sys=0 | e0004=1) = 3.904e-03

Eval P(sys | e0005=1)
P(sys=1 | e0005=1) = 9.984e-01
P(sys=0 | e0005=1) = 1.617e-03

Eval P(sys | e0006=1)
P(sys=1 | e0006=1) = 9.963e-01
P(sys=0 | e0006=1) = 3.702e-03

Eval P(sys | e0007=1)
P(sys=1 | e0007=1) = 9.960e-01
P(sys=0 | e0007=1) = 3.961e-03

Eval P(sys | e0008=1)
P(sys=1 | e0008=1) = 9.961e-01
P(sys=0 | e0008=1) = 3.910e-03

Eval P(sys | e0009=1)
P(sys=1 | e0009=1) = 9.960e-01
P(sys=0 | e0009=1) = 4.037e-03

Eval P(sys | e0010=1)
P(sys=1 | e0010=1) = 9.960e-01
P(sys=0 | e0010=1) = 3.965e-03

Eval P(sys | e0011=1)
P(sys=1 | e0011=1) = 9.963e-01
P(sys=0 | e0011=1) = 3.715e-03

Eval P(sys | e0012=1)
P(sys=1 | e0012=1) = 9.973e-01
P(sys=0 | e0